<a href="https://colab.research.google.com/github/prasanth30/DEVREV-RAG/blob/main/DPR_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
!pip install sentence-transformers

In [4]:

cd /content/drive/MyDrive/

/content/drive/MyDrive


In [5]:
!pip install farm-haystack[colab,inference,metrics]

In [6]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [7]:
from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

In [8]:
doc_dir = "/content/drive/MyDrive/"
train_filename = "new_T.json"
dev_filename = "new_D.json"

query_model = "facebook/dpr-question_encoder-single-nq-base"
passage_model = "facebook/dpr-ctx_encoder-single-nq-base"

save_dir = "../saved_models/dpr"


In [9]:
retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=256,
)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [10]:
retriever.train(
    data_dir=doc_dir,
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=1,
    batch_size=8,
    grad_acc_steps=8,
    save_dir=save_dir,
    evaluate_every=3000,
    embed_title=True,
    num_positives=1,
    num_hard_negatives=1,
)

INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: /content/drive/MyDrive/new_T.json 
INFO:haystack.modeling.data_handler.processor:Couldn't find /content/drive/MyDrive/new_T.json locally. Trying to download ...
INFO:haystack.modeling.data_handler.processor:downloading and extracting file MyDrive to dir /content/drive
ERROR:haystack.modeling.data_handler.processor:Cannot download MyDrive. Unknown data source.


FileNotFoundError: ignored

In [ ]:
import json

def check_dataset_format(dataset):
    for example in dataset:
        # Check if each example has the required keys
        required_keys = ["dataset", "question", "answers", "positive_ctxs", "negative_ctxs", "hard_negative_ctxs"]
        missing_keys = [key for key in required_keys if key not in example]
        if missing_keys:
            print(f"Missing keys in example: {missing_keys}")

        # Check if the types of values are correct
        if not isinstance(example["dataset"], str):
            print("Invalid 'dataset' type")

        if not isinstance(example["question"], str):
            print("Invalid 'question' type")

        if not isinstance(example["answers"], list) or not all(isinstance(ans, str) for ans in example["answers"]):
            print("Invalid 'answers' type")

        if not isinstance(example["positive_ctxs"], list) or not all(isinstance(ctx, dict) for ctx in example["positive_ctxs"]):
            print("Invalid 'positive_ctxs' type")

        if not isinstance(example["negative_ctxs"], list) or not all(isinstance(ctx, dict) for ctx in example["negative_ctxs"]):
            print("Invalid 'negative_ctxs' type")

        if not isinstance(example["hard_negative_ctxs"], list) or not all(isinstance(ctx, dict) for ctx in example["hard_negative_ctxs"]):
            print("Invalid 'hard_negative_ctxs' type")

        # Add similar checks for 'negative_ctxs' and 'hard_negative_ctxs'

# Load train dataset
train_file = "/content/drive/MyDrive/train_data.json"  # Update with the correct path
with open(train_file, "r") as f:
    train_dataset = json.load(f)

# Check train dataset format
check_dataset_format(train_dataset)

# Load dev dataset
dev_file = "/content/drive/MyDrive/dev_data.json"  # Update with the correct path
with open(dev_file, "r") as f:
    dev_dataset = json.load(f)

# Check dev dataset format
check_dataset_format(dev_dataset)


In [ ]:
train_file = "/content/drive/MyDrive/train_data.json"  # Update with the correct path
with open(train_file, "r") as f:
    train_dataset = json.load(f)
    print(len(train_dataset))